# Sequence to Sequence Learning with Neural Networks(NIPS 2014)
- RNN LSTM, Attention 등을 공부하는 과정에서 seq2seq 모델을 공부하게 되었다.
- seq2seq 모델은 전통적인 RNN 또는 LSTM 기반의 언어 모델과 달리, 인코더와 디코더를 사용하여 문장을 번역한다.
- 전통적인 언어 모델은 다음과 같은 대표적인 한계점이 존재한다.
    - 입력과 출력의 크기가 같다고 가정한다.
    - 또한, 예를 들어 한글과 영어의 경우 어순이 완전히 다르기 때문에 번역 시 문장 간의 관계를 찾는데 어려움이 따른다.
- seq2seq 모델은 인코더에서 고정된 크기의 context vector를 출력하고, context vector로부터 디코더가 번역한 결과를 추론하기
때문에 위와 같은 한계점을 극복할 수 있다.
- 이러한 seq2seq 모델을 아래의 링크를 참조하여 구현해보았다.  
  [papar: Sequence to Sequence Learning with Neural Networks(NIPS, 2014)](https://arxiv.org/abs/1409.3215)  
  [seq2seq review](https://www.youtube.com/watch?v=4DzKM0vgG1Y)

## 데이터 전처리(Prepocessing)
- spaCy 라이브러리:
    - 자연어 처리에서 많이 사용되는 파이썬 라이브러리
    - 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
    - 해당 라이브러리에서 영어(English)와 독일어(Deutsch) 전처리 모듈을 설치

In [ ]:
## Anaconda 프롬프트 상에서 실행함.
# %%capture
# !python -m spacy download en
# !python -m spacy download de

In [ ]:
import spacy

spacy_en = spacy.load('en') # 영어 토큰화
spacy_de = spacy.load('de') # 독일어 토큰화

In [ ]:
# 간단히 토큰화 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"index {i}: {token.text}")

index 0: I
index 1: am
index 2: a
index 3: graduate
index 4: student
index 5: .


- 영어 및 독일어 토큰화 함수 정의
    - 논문 상에서 입력에 사용하는 문장의 경우 문장의 뒤쪽부터 입력하는 것이 성능 향상에 매우 좋았다고 하였으므로, 
    입력으로 사용할 독일어의 경우 문장의 순서를 뒤집도록 하였다.

In [ ]:
# 독일어 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def de_tokenizer(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]
# 영어 문장을 토큰화 하는 함수
def en_tokenizer(text):
    return [token.text for token in spacy_en.tokenizer(text)]

- 필드(Field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시한다.
- 번역 목표:
    - 소스(SRC): 독일어
    - 목표(TRG): 영어

In [ ]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=de_tokenizer, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=en_tokenizer, init_token="<sos>", eos_token="<eos>", lower=True)

D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


- 대표적인 영어-독일어 번역 데이터셋인 __Multi30k__를 불러온다.
- 일반적으로 언어 번역 관련 논문에서는 수백만 개의 문장으로 데이터셋을 사용하지만, 실습에서는 이보다는 적은 수인 3만개의 문장을 사용한다.

In [ ]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [ ]:
print(f"train_dataset size: {len(train_dataset.examples)}")
print(f"valid_dataset size: {len(valid_dataset.examples)}")
print(f"test_dataset size: {len(test_dataset.examples)}")

train_dataset size: 29000
valid_dataset size: 1014
test_dataset size: 1000


In [ ]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[15])['src']) # 출력 결과를 보면, 입력의 순서가 뒤바뀌어 마침표가 가장 앞에 오는 것을 알
                                               # 수 있다.
print(vars(train_dataset.examples[15])['trg'])

['.', 'gugelhupf', 'einem', 'auf', 'puderzucker', 'streut', 'brille', 'und', 'oberteil', 'schwarzem', 'mit', 'frau', 'eine']
['a', 'lady', 'in', 'a', 'black', 'top', 'with', 'glasses', 'is', 'sprinkling', 'powdered', 'sugar', 'on', 'a', 'bundt', 'cake', '.']


- Field 객체의 build_vocab 메서드를 이용해 영어와 독일어의 단어 사전을 생성한다.
- 이 때 min_freq 옵션을 사용해 최소 2번 이상 등장한 단어만 선택한다.

In [ ]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len of SRC: {len(SRC.vocab)}")
print(f"len of TRG: {len(TRG.vocab)}")

len of SRC: 7854
len of TRG: 5893


- 위의 코드를 통해 단어 사전이 생성되면 SRC.vocab.stoi는 어휘에 해당하는 토큰을 키로, 
관련된 색인을 값으로 가지는 dictionary가 된다.

In [ ]:
print(TRG.vocab.stoi['abcabc']) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


- 한 문장에 포함된 단어가 연속적으로 LSTM에 입력되어야 한다.
- 또한, 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사할 때 효율적으로 학습시킬 수 있다.
- 이를 위해 BucketIterator를 사용하는데, BucketIterator는 배치 크기(batch_size)에 따라 각각의 배치가 길이가 비슷한 문장으로
구성되도록 한다.

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 data_loader의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_dataset, valid_dataset, test_dataset),
                                                                     batch_size=BATCH_SIZE, device=device)

D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


- 아래는 첫 번째 배치의 첫번째 문장 정보를 출력하는 코드이다.
- 첫번째 배치의 길이는 28이므로, 첫번째 배치의 모든 문장의 길이가 28이 되도록 패딩된 것을 알 수 있다.
- 문장의 길이에 따라 패딩이 들어가므로, 아래에서 1로 된 값들은 패딩되어 추가된 부분을 의미한다.
- 또한 index 0에는 2, index 16에는 3이 입력되어 있는 것을 볼 수 있는데, 이는 각각 "<sos>", "<eos>"에 해당하는 단어 사전의 색인값이다.

In [ ]:
for i, batch in enumerate(train_iterator):
    # 각 배치는 src 및 trg 속성을 가진다.
    src = batch.src
    trg = batch.trg
    
    print(f"Shape of first batch: {src.shape}")
    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"index {i}: {src[i][0].item()}")
    # 첫번째 배치만 확인    
    break

Shape of first batch: torch.Size([28, 128])
index 0: 2
index 1: 4
index 2: 1850
index 3: 281
index 4: 10
index 5: 2672
index 6: 6
index 7: 12
index 8: 505
index 9: 100
index 10: 207
index 11: 24
index 12: 11
index 13: 156
index 14: 13
index 15: 5
index 16: 3
index 17: 1
index 18: 1
index 19: 1
index 20: 1
index 21: 1
index 22: 1
index 23: 1
index 24: 1
index 25: 1
index 26: 1
index 27: 1


D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


## 인코더(Encoder) 아키텍쳐
- 인코더는 주어진 소스 문장을 context vector로 인코딩한다.
- 인코더와 디코더는 디코더에는 FC층이 추가된 다는 것 빼고는 거의 유사한 구조를 가진다.
- LSTM은 hidden state와 cell state를 리턴한다.
- 하이퍼 파라미터는 아래와 같다.
    - __input_dim__: 하나의 단어에 대한 원핫 인코딩 차원
    - __embed_dim__: 임베딩(embedding) 차원
    - __hidden_dim__: 히든 상태(hidden state) 차원
    - __n_layers__: LSTM 층 개수
        - 기존 논문에서는 LSTM을 4번 중첩하여 사용하였지만, 이번 실습에서는 LSTM을 2번 중첩하여 사용한다.
    - __dropout_ratio__: 드롭아웃 비율(일반적으로 0.5)

In [ ]:
import torch.nn as nn

# 인코더 아키텍쳐 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        # 임베딩(embedding)은 원-핫 인코딩을 특정 차원의 임베딩으로 매핑하는 층
        self.embedding = nn.Embedding(input_dim, embed_dim)
        # LSTM 층
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)
        
    # 인코더는 소스 문장을 입력으로 받아 context vector를 반환
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 단어 사전 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        # output: [단어 개수, 배치 크기, 히든 차원] -> 현재 단어의 출력 정보 
        # 인코더에서는 output은 사용되지 않고, 문맥 정보를 담은 hidden과 cell만 사용한다.
        # hidden: [층 개수, 배치 크기, 히든 차원] -> 현재까지의 모든 단어의 단기 기억 정보
        # cell: [층 개수, 배치 크기, 히든 차원] -> 현재까지의 모든 단어의 장기 기억 정보
        
        # context vector 반환
        return hidden, cell

## 디코더(Decoder) 아키텍쳐
- 인코더와 아키텍쳐는 유사하다.
- 주어진 context vector를 타겟 문장으로 디코딩한다.
- LSTM은 hidden state와 cell state를 반환한다.
- 하이퍼 파라미터는 아래와 같다.
    - __output_dim__: 하나의 단어에 대한 원핫 인코딩 차원
    - __embed_dim__: 임베딩(embedding) 차원
    - __hidden_dim__: 히든 상태(hidden state) 차원
    - __n_layers__: LSTM 층 개수
    - __dropout_ratio__: 드롭아웃 비율

In [ ]:
# 디코더 아키텍쳐 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, emded_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        # 임베딩은 원-핫 인코딩 말고 특정 차원의 임베딩으로 매핑하는 층
        self.embedding = nn.Embedding(output_dim, emded_dim)
        # LSTM 층
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(emded_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        # FC 층(인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)
    # forward 함수는 단어를 하나씩 입력하고, 
    # 출력 단어, 현재까지의 모든 단어의 장/단기 기억 정보를 리턴한다.
    def forward(self, input, hidden, cell):
        # input: [배치 크기] -> 단어의 개수는 항상 1개 이도록 구현
        # hidden: [층 개수, 배치 크기, 히든 차원]
        # cell = context: [층 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]
        
        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [단어 개수 = 1, 배치 크기, 히든 차원] -> 현재 단어의 출력 정보
        # hidden: [층 개수, 배치 크기, 히든 차원] -> 현재까지의 모든 단어의 단기 기억 정보
        # cell: [층 개수, 배치 크기, 히든 차원] -> 현재까지의 모든 단어의 장기 기억 정보
        
        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction: [배치 크기, 출력 차원]
        
        # (현재 출력 단어, 현재까지의 모든 단어의 단기 기억 정보, 현재까지의 모든 단어의 장기 기억 정보)를 리턴한다.
        return prediction, hidden, cell

## Seq2seq 아키텍쳐
- 앞서 정의한 인코더와 디코더를 가지고 있는 하나의 아키텍쳐이다.
    - 인코더: 주어진 소스 문장을 context vector로 인코딩한다.
    - 디코더: 주어진 context vector를 타겟 문장으로 디코딩한다.
    - 디코더는 한 단어씩 입력하여 한 번씩 결과를 출력한다.
- Teacher Forcing 기법: 디코더의 예측(prediction)을 다음 입력으로 사용하지 않고, 실제 타겟 출력(ground-truth)을 
    다음 입력으로 사용하는 기법이다.
    - 이 방법을 사용할 때 훨씬 좋은 성능이 나온다고 한다.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    # 학습할 때는 완전한 형태의 소스 문장, 타켓 문장, teacher_forcing_ratio를 입력한다.
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        
        # 먼저 인코더를 거쳐 context vector를 도출
        hidden, cell = self.encoder(src)
        # 디코더의 최종 결과를 담을 텐서 객체 생성
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # 첫 번째 입력은 항상 "<sos>" 토큰
        input = trg[0, :] # 배치 내 각 문장의 첫번째 단어("<sos>")를 입력
        
        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t] = output     # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 배치 단위로 각 단어에서 가장 확률이 높은 단어의 인덱스 추출
                                    # output([배치 크기, 출력 차원])은 각 단어의 원-핫 인코딩 형태
            
            # teacher_forcing_ratio: 학습 시 실제 목표 출력(ground truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에 삽입
            
        return outputs

## 학습(Training)
- 하이퍼 파라미터 설정 및 모델 초기화

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [ ]:
# 인코더와 디코더 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# seq2seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

- 논문의 내용대로 $\mu$(-0.08, 0.08)의 값으로 모델의 가중치 초기화

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

- 학습 및 평가 함수 정의

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 손실값을 계산할 때 뒷 부분의 패딩(padding)에 대해서는 값 무시
# 애초에 단어가 존재하지 않기 때문에 패딩을 해당 위치에 채워넣은 것이기 때문
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
from tqdm import tqdm

# 모델 학습 함수 정의
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for batch in tqdm(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어 개수 - 1) * 배치 크기, output_dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어 개수 - 1) * 배치 크기]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 그레디언트 계산
        
        # 그레디언트 clipping 진행(논문 내용)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

In [ ]:
# 모델 평가 함수 정의
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            # 평가할 때 teacher forcing은 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output: [(출력 단어 개수 - 1) * 배치 크기, 출력 차원]
            trg = trg[1:].view(-1)
            # trg: [(타겟 단어의 개수 - 1) * 배치 크기]
            
            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)
            
            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
        return epoch_loss / len(iterator)

- 학습 및 검증 진행
    - 학습 횟수(epoch): 20

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')
        
    print(f"Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}")
    print(f"\tValidation loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}")

  0%|                                                                                          | 0/227 [00:00<?, ?it/s]D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
  0%|                                                                                          | 0/227 [00:00<?, ?it/s]

Epoch: 01 | Time: 10m 33s
	Train loss: 4.396 | Train PPL: 81.162
	Validation loss: 4.703 | Validation PPL: 110.330


  0%|                                                                                          | 0/227 [00:00<?, ?it/s]

Epoch: 02 | Time: 10m 28s
	Train loss: 4.116 | Train PPL: 61.297
	Validation loss: 4.547 | Validation PPL: 94.377


  0%|                                                                                          | 0/227 [00:00<?, ?it/s]

...

Epoch: 18 | Time: 9m 48s
	Train loss: 2.404 | Train PPL: 11.065
	Validation loss: 3.727 | Validation PPL: 41.568


  0%|                                                                                          | 0/227 [00:00<?, ?it/s]

Epoch: 19 | Time: 9m 45s
	Train loss: 2.300 | Train PPL: 9.976
	Validation loss: 3.731 | Validation PPL: 41.717


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [09:48<00:00,  2.59s/it]


Epoch: 20 | Time: 9m 51s
	Train loss: 2.268 | Train PPL: 9.656
	Validation loss: 3.713 | Validation PPL: 40.971


In [ ]:
model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(f"Test loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}")

D:\Anaconda\envs\torch1.6\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test loss: 3.698 | Test PPL: 40.375


## 나만의 데이터로 모델 사용해보기

In [ ]:
# 번역(translate) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드
    
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    
    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")
    
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    
    # 인코더에 소스 문장을 넣어 context vector 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    
    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록 
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기
        
        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        
    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[vocab_idx] for vocab_idx in trg_indexes]
    
    # 첫번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

- 출력 결과를 보면, 소스 문장을 입력하였을 때 타겟 문장과 완전히 똑같지는 않지만, 비슷한 결과를 내는 것을 알 수 있다.

In [ ]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print('모델 출력 결과:', " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', '.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine', '<eos>']
소스 문장 인덱스: [2, 4, 88, 20, 200, 780, 19, 565, 624, 70, 134, 10, 364, 8, 3]
모델 출력 결과: a mother and her son are in in a public park . <eos>


- 새로운 입력을 입력하였을 때에는 제대로된 결과를 내지는 못했지만, 실습을 참고한 코드에서는 정확한 결과를 낸 것을 확인하였다.
- 데이터셋의 크기가 크지 않고 모델이 아직은 충분히 학습되지 못하여 이러한 결과를 낸 것으로 보인다.

In [ ]:
src = de_tokenizer("Guten abend.")

print(f'소스 문장: {src}')
print('모델 출력 결과:', " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'abend', 'Guten']
전체 소스 토큰: ['<sos>', '.', 'abend', 'guten', '<eos>']
소스 문장 인덱스: [2, 4, 1163, 3798, 3]
모델 출력 결과: <unk> . <eos>
